Face recognition task.
0) First we create vectorized database of faces using facenet architecture.

Recognition pipeline:
1) Every frame from video stream is passing through mtcnn face detector to get cropped face images,
2) Faces with shape less than 16x16 are excluded,
3) Faces with shape less than 32x32 are passing through SRGAN to get better quality,
4) Every face image is passing through head pose estimator to exclude faces rotated on big angle
5) Face images are passing through facenet to get vectorized representation of faces
6) Finally every vectorized face compares with faces in database using cosine distance. 
7) If distance between passing vectorized face and face in database is less than threshold we count passing vectorized face as recognized

In [2]:
from mtcnn.mtcnn import MTCNN
import pickle
import cv2
import numpy as np
import face_recognition 
import matplotlib.pyplot as plt
import dlib
from imutils import paths
import os
import PIL
from scipy.spatial.distance import cosine 
%matplotlib inline
import time
import os
import keras.backend as K
from keras.models import load_model
mtcnn_detector = MTCNN()


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices())


# uploading Facenet

from keras.models import load_model
model_path = '/facenet_keras.h5'
model = load_model(model_path)


# uploading headpose

def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

head_pose = load_model('/media/aevdakimov/HDisk/Arthur/HeadPose/weights-improvement-40-0.10_FINAL_normed.hdf5',
                      custom_objects={'euclidean_distance_loss':euclidean_distance_loss})



['/device:CPU:0', '/device:GPU:0', '/device:GPU:1']


/home/aevdakimov/anaconda3/envs/myenv/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
# SRGAN

from keras.applications.vgg19 import VGG19
from keras.models import Model


def vgg_loss(y_true, y_pred):
    
    vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=image_shape)
    vgg19.trainable = False
    for l in vgg19.layers:
        l.trainable = False
    loss_model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block5_conv4').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))

def normalize(input_data):

    return (input_data.astype(np.float32) - 127.5)/127.5 
    
def denormalize(input_data):
    input_data = (input_data + 1) * 127.5
    return input_data.astype(np.uint8) 


image_shape=(128,128,3)
generator = load_model('/gen_model_32_150.h5',
                         custom_objects={'vgg_loss':vgg_loss} )



In [3]:
# ENCODING DATABASE using FACENET 
 
# path to face database
data_path = '/employees'


imagePaths = list(paths.list_images(data_path))


knownEncodings = []
knownNames = []
faces = []
encodings=[]

for (i, imagePath) in enumerate(imagePaths):

    name = imagePath.split(os.path.sep)[-2]
    
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
    rgb_cropped = PIL.Image.fromarray(rgb)
    rgb_cropped = rgb_cropped.resize((160,160))
    rgb_cropped = np.array(rgb_cropped)
    rgb_cropped = rgb_cropped/255.
    
    encoding = model.predict(np.expand_dims(rgb_cropped, axis=0)) # encoding faces
    encodings.append(encoding)
    faces.append(rgb_cropped)
    knownNames.append(name)
    knownEncodings.append(encoding)
    
data = {"encodings": knownEncodings, "names": knownNames, 'faces':faces}
f = open('/test_facenet_1.pickle', "wb")
f.write(pickle.dumps(data))
f.close()

In [4]:
# Detecting faces with MTCNN and use Descriptor FACENET 
# 


import face_recognition
import argparse
import pickle
import cv2
import PIL
from mtcnn.mtcnn import MTCNN
import numpy as np
import dlib
from imutils import paths
import os
from scipy.spatial.distance import cosine 
%matplotlib inline
import time


def face_distance(face_encodings, face_to_compare): 
    # Function to compute cosine similarity between faces
    
    if len(face_encodings) == 0:
        return np.empty((0))

    dist = [cosine(i, face_to_compare) for i in face_encodings]
    return dist

def compare_faces(known_face_encodings, face_encoding_to_check, thresh=0.55):
    # Function to return faces below threshold
    return np.array(face_distance(known_face_encodings, face_encoding_to_check)) <= thresh, np.array(face_distance(known_face_encodings, face_encoding_to_check))


# Uploading Data Base with vectorized faces
data = pickle.loads(open('/facenet_1.pickle',
                         "rb").read())
    
def RECOGNIZER(frame, data, identifiers):
    
    head_thresh = 0.3 # threshold for head
    image = frame
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    box_mtcnn = mtcnn_detector.detect_faces(rgb)
    boxes = []
    encodings =[]
    
    for i in box_mtcnn:
        
        if max(i['box'][3],i['box'][2])<16: # minimal size check 
            continue
        
        
        if max(i['box'][3],i['box'][2])>=16 and max(i['box'][3],i['box'][2])<=32: # SRGAN size check
            
            L = max(i['box'][3],i['box'][2]) - min(i['box'][3],i['box'][2])
            rgb_cropped = rgb[i['box'][1]:i['box'][1]+i['box'][3], i['box'][0]-int(L/2):i['box'][0]+i['box'][2]+int(L/2)]
            rgb_cropped = PIL.Image.fromarray(rgb_cropped)
            rgb_cropped = rgb_cropped.resize((32,32))
            rgb_cropped = np.array(rgb_cropped)            
            cv2.imshow('face_lr',rgb_cropped)            
            rgb_cropped = denormalize(generator.predict(np.reshape(normalize(np.array(rgb_cropped)), (1,32,32,3))))[0] #SRGAN            
            rgb_cropped = PIL.Image.fromarray(rgb_cropped)
            rgb_cropped = rgb_cropped.resize((160,160))
            rgb_cropped = np.array(rgb_cropped)
            rgb_cropped = rgb_cropped/255    
            cv2.imshow('face_sr',rgb_cropped)           
            top = i['box'][1]
            right = i['box'][0] + i['box'][2]
            bottom = i['box'][1] + i['box'][3] 
            left = i['box'][0]    
            boxes.append((top,right,bottom,left))     
            encoding = model.predict(np.expand_dims(rgb_cropped, axis=0)) #add encoding
            encodings.append(encoding)          
            
        if max(i['box'][3],i['box'][2])>=32: 
            try:
                L = max(i['box'][3],i['box'][2]) - min(i['box'][3],i['box'][2])
                rgb_cropped = rgb[i['box'][1]:i['box'][1]+i['box'][3], i['box'][0]-int(L/2):i['box'][0]+i['box'][2]+int(L/2)]
           
                rgb_cropped = PIL.Image.fromarray(rgb_cropped)
                rgb_cropped = rgb_cropped.resize((128,128))
                rgb_cropped = np.array(rgb_cropped)
                
                head = head_pose.predict(np.expand_dims(rgb_cropped/255., axis = 0)) # headpose estimation
                
                if np.absolute(head[0][0] + head[0][1]) >= head_thresh: # headpose filter
                    continue
                
                rgb_cropped = PIL.Image.fromarray(rgb_cropped)
                rgb_cropped = rgb_cropped.resize((160,160))
                rgb_cropped = np.array(rgb_cropped)
                rgb_cropped = rgb_cropped/255.
        
                top = i['box'][1]
                right = i['box'][0] + i['box'][2]
                bottom = i['box'][1] + i['box'][3] 
                left = i['box'][0]
            
                boxes.append((top,right,bottom,left))
                
                encoding = model.predict(np.expand_dims(rgb_cropped, axis=0))
                encodings.append(encoding)
                
            except Exception as e:
                print(e)
    
    names = []
    
    for encoding in encodings:
        matches, dist = compare_faces(data["encodings"], np.reshape(np.array(encoding), (128)))
        name = "Unknown"
        
        print(matches)
        print(dist)
        if True in matches:
            print(matches)
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {} 
            
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
                
                
            name = max(counts, key=counts.get)
            print(name)
        names.append(name) 
        identifiers[name] = identifiers.get(name, 0) + 1
        
        print(identifiers)
    print(names)
    
    for ((top, right, bottom, left), name) in zip(boxes, names):
            
          
        if (name!="Unknown") and (identifiers[name] >= 5): # If person was recognized on 5 frames, then count recognition completed successfuly
            cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2) # Box printing
            y = top - 15 if top - 15 > 15 else top + 15
            cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
            identifiers[name] = 0 # refresh counnter
            print("PERSON___",name,"___RECOGNIZED")
        
        else:
                                                           # Box printing
            cv2.rectangle(image, (left, top), (right, bottom), (255, 0, 0), 2)
            y = top - 15 if top - 15 > 15 else top + 15
            cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)
            
            
        if (name=='Unknown') and (identifiers[name]>=5): # If person was recognized on 5 frames, as "Unnkonw" then count recognition not completed successfuly
            cv2.rectangle(image, (left, top), (right, bottom), (0, 0, 255), 2)
            y = top - 15 if top - 15 > 15 else top + 15
            cv2.putText(image, 'Unknown', (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
            identifiers[name] = 0
            print("UNKONW____________________ DETECTED")
            
    return(image, identifiers)


In [6]:
# Launcher 
import numpy as np 
import cv2   


cap = cv2.VideoCapture(0)

cap.set(3,1280)
cap.set(4,720)
  
fourcc = cv2.VideoWriter_fourcc(*'MJPG') 
out = cv2.VideoWriter('output.avi', fourcc, 30.0, (1280, 720)) 

identifiers = {}
 
while(True): 
    ret, frame = cap.read()      
    hsv = frame    
    hsv,identifiers = RECOGNIZER(hsv, data, identifiers)
    
    out.write(hsv)       
    cv2.imshow('frame', hsv) 
     
    if cv2.waitKey(1) & 0xFF == ord('a'): 
        break
          
cap.release() 
  
out.release()  
   
cv2.destroyAllWindows() 

[]
[]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
[1.07791519 0.74397933 1.05578991 0.81979184 0.92867769 0.8009315
 1.01764599 0.72514999 0.61978891 0.77295461 0.73587006 0.99443511
 0.86225705 1.10460027 0.80481067 0.91226415 0.85203797 0.79764742
 0.72423053 0.87887236 0.99223944 0.80632094 0.82317175 0.84924778
 0.96851693 1.12668741 0.98385675 0.88706495 0.97347323 0.79065017
 0.87901608 0.85769729 0.96407621 0.88170785 0.95290553 0.84738782
 0.9069186  0.98229798 0.69040328 0.82233165]
{'Unknown': 1}
['Unknown']
[]
[]
[]
[]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
[0.95916299 0.8928

[]
[]
[False False False False False False False  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
[0.82171808 0.95174272 0.76348247 1.03859367 0.72664753 0.88095441
 0.96960883 0.29278076 0.84423564 0.74698663 0.79684134 0.68989822
 0.71024522 0.63061404 0.539637   1.02303964 0.93577959 0.92731231
 0.87649467 0.96243514 0.88811725 0.80604772 1.01131999 0.82806434
 1.02682421 0.87560133 0.71693394 1.01570675 0.86014915 0.71224555
 0.78283131 0.77666965 0.98689927 0.91325102 0.82342444 0.92812
 0.96857994 0.92438309 0.90642615 1.00218265]
[False False False False False False False  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
Evdakimov
{'Unknown': 0, 'Evdakimov': 1}
['Evdakimov']
[]
[

[False False False False False False False  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
[0.91865484 0.97158218 0.77358103 1.06577244 0.72927147 0.78635998
 0.9709844  0.36391032 0.80545765 0.68673193 0.78439717 0.80527337
 0.75221311 0.64127195 0.48190278 1.07614327 0.94696629 0.89028388
 0.85689606 0.96491528 0.92698268 0.84683058 1.07245558 0.83248498
 1.00597758 0.88330662 0.72048038 1.02351198 0.93374127 0.79356028
 0.80470179 0.84355663 1.02758513 0.91856067 0.8541653  0.94289013
 1.02643899 0.91024596 0.87883036 1.08657789]
[False False False False False False False  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
Evdakimov
{'Unknown': 1, 'Evdakimov': 4}
['Evdakimov']
[False 

[]
[]
[]
[]
[False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
[0.84129238 1.05764077 0.80288027 1.05308133 0.81743039 0.77510947
 0.89061851 0.3577081  0.94601405 0.8185138  0.74902606 0.8019989
 0.81295985 0.7872732  0.55553153 0.96136046 1.0002094  0.92302853
 0.86802228 1.02817131 0.92047063 0.88560542 0.97812317 0.80096513
 1.0706878  0.86606134 0.84300485 1.02114471 0.9638932  0.83354644
 0.86878204 0.69572875 0.97413886 0.98189506 0.847341   0.93227355
 1.00468729 0.86460656 0.83608148 1.09912118]
[False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
Evdakimov
{'Unknown': 3, 'Evdakimov': 1}
['Evdakimo

[False False False False False False False  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
[0.88148531 1.00313094 0.79138058 1.02939255 0.73498413 0.78318092
 0.97395608 0.36843175 0.83815682 0.66455269 0.83495878 0.7613757
 0.7477102  0.71497515 0.511282   1.12658818 0.91140555 0.91068275
 0.7327632  0.97643894 0.91787523 0.84858416 1.05902322 0.92750309
 1.02235628 0.91361801 0.78836262 1.05090851 0.8738801  0.74433208
 0.8126428  0.8594719  1.06873427 0.9631824  0.78848884 0.8519583
 1.01914907 0.9544116  0.99751263 1.11383276]
[False False False False False False False  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
Evdakimov
{'Unknown': 3, 'Evdakimov': 5}
['Evdakimov']
PERSON___

[False False False False False False False  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
[0.85860603 1.0173758  0.77002095 1.02401931 0.72492051 0.76539719
 0.96768557 0.39440286 0.86742394 0.67771411 0.84133683 0.78310931
 0.73603654 0.67649963 0.4795478  1.07904121 0.8961674  0.89913469
 0.74599156 0.95792113 0.90075929 0.86485136 1.06944133 0.93027752
 0.98012318 0.87369514 0.7847604  1.04682523 0.84635343 0.74860495
 0.81828521 0.8723391  1.08130256 0.95252056 0.78349136 0.85446478
 1.00241374 0.95944434 0.98293826 1.13032109]
[False False False False False False False  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
Evdakimov
{'Unknown': 3, 'Evdakimov': 5}
['Evdakimov']
PERSON_

[False False False False False False False  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
[0.86052136 0.97364547 0.79263595 0.99850602 0.66498467 0.75528704
 0.95059382 0.32019114 0.84563009 0.70258102 0.77347822 0.82486412
 0.77785669 0.71054375 0.47421896 1.01289629 1.00146793 0.88086151
 0.77372824 0.98826848 0.89281618 0.82828149 0.97240202 0.89277294
 1.04768509 0.87170199 0.75286666 1.00970222 0.87052201 0.77506019
 0.78977895 0.73783734 1.0584937  0.91327418 0.85486098 0.82912223
 0.9966725  0.95293953 0.94865859 1.14088222]
[False False False False False False False  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
Evdakimov
{'Unknown': 3, 'Evdakimov': 4}
['Evdakimov']
[False 

[False False False False False False False  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
[0.74194303 1.0014443  0.69609508 0.96866509 0.72200811 0.83113262
 0.90891648 0.42788321 0.92265176 0.77219489 0.81588016 0.83807985
 0.78083038 0.63862339 0.51829264 0.98839404 0.97174293 0.88608413
 0.88356846 0.99971686 0.90302876 0.80302608 0.94000404 0.79523981
 1.12001584 0.88801175 0.72992307 1.03864105 0.884773   0.85295926
 0.81168625 0.71515697 1.06121151 0.92018285 0.87325658 1.01003875
 1.00288374 0.88131902 0.87437437 1.08070968]
[False False False False False False False  True False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
Evdakimov
{'Unknown': 3, 'Evdakimov': 4}
['Evdakimov']
[False 